In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [47]:
!ls -lh

total 20G
drwxrwxr-x 4 core core  4.0K Aug  5 16:44 catboost_info
-rw-rw-r-- 1 core core   852 Aug  4 15:31 cat_feature_index.c1a4dabd-715a703f-75517141-d81266d1.tmp
-rw-rw-r-- 1 core core   852 Aug  4 15:04 cat_feature_index.e36fdba6-6cd399f6-229ea45f-ea07b906.tmp
-rw-rw-r-- 1 core core  1.3M Jul 25 11:20 eda.ipynb
-rw-rw-r-- 1 core core  370K Jul 30 17:58 Is-classifier-6-eda.ipynb
-rw-rw-r-- 1 core core  377K Jul 29 05:21 Is-detect-42-eda.ipynb
drwxrwxr-x 2 core core  4.0K Aug  6 15:42 lb18901
drwxrwxr-x 2 core core  4.0K Aug  6 15:42 lb19252
-rw-rw-r-- 1 core core  182M Jul 30 16:25 merge_0724_2.csv.zip
-rw-rw-r-- 1 core core  164M Jul 26 05:14 merge_0726_1.csv.zip
-rw-rw-r-- 1 core core  148M Jul 28 07:53 merge_0728_1.csv.zip
-rw-rw-r-- 1 core core  219M Aug  6 15:45 merge_0806_1.csv.zip
-rw-rw-r-- 1 core core  229M Aug  6 15:52 merge_0806_2_fillna.csv.zip
-rw-rw-r-- 1 core core  283M Aug  6 16:10 merge_0806_3_fillna_top100tuple.csv.zip
-rw-rw-r-- 1 core core   45K Aug  6 16:10 mer

In [48]:
csv_files = [
            'sub_sort_100_0806_2_remove_none_lb19252.csv',
            'sub_is_42_0726_2.csv',
            #'sub_retina_is_42_0801.csv',
            #'sub_101_0-50_100boxes_0709_1.csv', 
             #'sub_101_400-500_100boxes_0711_1.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [49]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [15]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...
2,7540c162180973b1,
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...


In [50]:
#keep top 100
dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [51]:
dfs[1].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.3806741 /m/04bcr3 0.0031277 0.7127081 0.4623...
1,763c72626bf24534,0.7894505 /m/04bcr3 0.6918395 0.8270421 0.9988...
2,7540c162180973b1,0.0402823 /m/04bcr3 0.3689175 0.6536201 0.6134...
3,c4e522dce52cb02f,0.7901130 /m/04bcr3 0.8337175 0.6788673 0.9917...
4,cafcd9496c808085,0.1362440 /m/04dr76w 0.0390787 0.0196190 0.961...


In [52]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [53]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

16700 5602


In [54]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...
2,7540c162180973b1,0.0402823 /m/04bcr3 0.3689175 0.6536201 0.613...
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...


In [55]:
dfs[0].to_csv('merge_0806_4_fillna_top100tuple.csv.zip', index=False, compression='zip')

In [57]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f merge_0806_4_fillna_top100tuple.csv.zip -m "submit"

100%|████████████████████████████████████████| 196M/196M [00:15<00:00, 13.1MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [ ]:
!ls

In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [102]:
dfs = [tmp1, tmp2, tmp3]
x = None
for d in dfs:
    if x is None:
        x = d.t
    else:
        x = x.str.cat(d.t, sep=' ')

#x = tmp1.t1.str.cat(tmp2.t2, sep=' ').str.cat(tmp3.t3, sep=' ')
print(x)
tmp1.t = x
tmp1.head()

0    Geeks a a1
1      For b b1
2     Geeks c c
3        is d d
4    portal e e
5       for f f
6     Geeks g g
Name: t, dtype: object


,t
0,Geeks a a1
1,For b b1
2,Geeks c c
3,is d d
4,portal e e


In [25]:
df0 = dfs[0]
df0.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...
2,7540c162180973b1,
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...


In [26]:
df0['dets'] = df0.PredictionString.map(lambda x: x.split(' '))
df0.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...,"[0.500607, /m/04yx4, 0.3793392, 0.2354858, 0.7..."
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...,"[0.8031, /m/04yx4, 0.7241355, 0.5230734, 0.949..."
2,7540c162180973b1,,[]
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...,"[0.7345, /m/04yx4, 0.8538917, 0.3660534, 0.971..."
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...,"[0.279052, /m/01_5g, 0.7975672, 0.0272237, 0.9..."


In [27]:
df0['num'] = df0.dets.map(lambda x: len(x))
df0.head()

,ImageId,PredictionString,dets,num
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...,"[0.500607, /m/04yx4, 0.3793392, 0.2354858, 0.7...",2328
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...,"[0.8031, /m/04yx4, 0.7241355, 0.5230734, 0.949...",336
2,7540c162180973b1,,[],1
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...,"[0.7345, /m/04yx4, 0.8538917, 0.3660534, 0.971...",3048
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...,"[0.279052, /m/01_5g, 0.7975672, 0.0272237, 0.9...",960


In [35]:
df0.dets = df0.dets.map(lambda x: x[:1200])
df0.head()

,ImageId,PredictionString,dets,num
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...,"[0.500607, /m/04yx4, 0.3793392, 0.2354858, 0.7...",2328
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...,"[0.8031, /m/04yx4, 0.7241355, 0.5230734, 0.949...",336
2,7540c162180973b1,,[],1
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...,"[0.7345, /m/04yx4, 0.8538917, 0.3660534, 0.971...",3048
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...,"[0.279052, /m/01_5g, 0.7975672, 0.0272237, 0.9...",960


In [36]:
df0['num'] = df0.dets.map(lambda x: len(x))
df0.head()

,ImageId,PredictionString,dets,num
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...,"[0.500607, /m/04yx4, 0.3793392, 0.2354858, 0.7...",1200
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...,"[0.8031, /m/04yx4, 0.7241355, 0.5230734, 0.949...",336
2,7540c162180973b1,,[],1
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...,"[0.7345, /m/04yx4, 0.8538917, 0.3660534, 0.971...",1200
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...,"[0.279052, /m/01_5g, 0.7975672, 0.0272237, 0.9...",960


In [37]:
df0['PredictionString'] = df0.dets.map(lambda x: ' '.join(x))
df0.head()

,ImageId,PredictionString,dets,num
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...,"[0.500607, /m/04yx4, 0.3793392, 0.2354858, 0.7...",1200
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...,"[0.8031, /m/04yx4, 0.7241355, 0.5230734, 0.949...",336
2,7540c162180973b1,,[],1
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...,"[0.7345, /m/04yx4, 0.8538917, 0.3660534, 0.971...",1200
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...,"[0.279052, /m/01_5g, 0.7975672, 0.0272237, 0.9...",960


In [38]:
df0.drop(['dets', 'num'], axis=1).head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500607 /m/04yx4 0.3793392 0.2354858 0.751103...
1,763c72626bf24534,0.8031 /m/04yx4 0.7241355 0.5230734 0.9497714 ...
2,7540c162180973b1,
3,c4e522dce52cb02f,0.7345 /m/04yx4 0.8538917 0.3660534 0.9713876 ...
4,cafcd9496c808085,0.279052 /m/01_5g 0.7975672 0.0272237 0.996426...


In [39]:
dfs[0] = df0

In [45]:
df0.iloc[0].PredictionString

'0.500607 /m/04yx4 0.3793392 0.2354858 0.7511032 0.9924041 /m/0hg7b 0.3797071 0.3322252 0.4894319 0.5877937 holds 0.479446 /m/04yx4 0.0087893 0.1316144 0.3998478 0.9693565 /m/01mzpv 0.0198562 0.6855568 0.4241981 0.9958978 on 0.356773 /m/04yx4 0.0087893 0.1316144 0.3998478 0.9693565 /m/0hg7b 0.3797071 0.3322252 0.4894319 0.5877937 holds 0.330706 /m/04yx4 0.3793392 0.2354858 0.7511032 0.9924041 /m/04bcr3 0.0107101 0.658406 0.4643026 0.9898675 at 0.32873 /m/04yx4 0.0087893 0.1316144 0.3998478 0.9693565 /m/04bcr3 0.0107101 0.658406 0.4643026 0.9898675 at 0.30718 /m/04yx4 0.3793392 0.2354858 0.7511032 0.9924041 /m/01mzpv 0.3951311 0.3523343 0.9539606 0.9940798 on 0.284311 /m/01bl7v 0.8140764 0.133122 0.9968848 0.9380613 /m/01mzpv 0.7079747 0.4603541 0.9764486 0.9949058 on 0.261801 /m/04yx4 0.0707568 0.1497036 0.9208518 0.9984375 /m/0hg7b 0.3797071 0.3322252 0.4894319 0.5877937 holds 0.247956 /m/04yx4 0.3793392 0.2354858 0.7511032 0.9924041 /m/04bcr3 0.3679503 0.7281674 0.4645532 0.8945925 a